---
layout: default
title: karplus strong
author: spielhuus
tags: documentation
---

# Karplus-Strong Synthesis

calculating frequencies from midi notes:

In [ ]:
def make_sine_wavetable(n_samples, amps, phases, freqs):
    """Makes a wavetable from a sum of sines."""
    t = np.linspace(0, 1, num=n_samples)
    wavetable = np.zeros_like(t)
    for amp, phase, freq in zip(amps, 
                                phases,
                                freqs):
        wavetable += amp * np.sin(np.sin(2 * np.pi * freq * t + phase)) + \
                         amp / 2 * np.sin(np.sin(2 * np.pi * 2 * freq * t + phase))
    return wavetable

In [ ]:
def make_triangle_wavetable():
    t = np.linspace(0, 1, num=n_samples)
    
    return wavetable

In [ ]:
import numpy as np
import matplotlib.pyplot as plot
from IPython.display import Audio

In [ ]:
fs = 8000
wavetable_size = fs // 55
wavetable = (2 * np.random.randint(0, 2, wavetable_size) - 1).astype(np.float)

calculate sine wave with cmsis dsp. 


In [ ]:
plot.plot(wavetable)


In [ ]:
Audio(wavetable, rate=fs)

In [ ]:
def karplus_strong(wavetable, n_samples):
    """Synthesizes a new waveform from an existing wavetable, modifies last sample by averaging."""
    samples = []
    current_sample = 0
    previous_value = 0
    while len(samples) < n_samples:
        wavetable[current_sample] = 0.5 * (wavetable[current_sample] + previous_value)
        samples.append(wavetable[current_sample])
        previous_value = samples[-1]
        current_sample += 1
        current_sample = current_sample % wavetable.size
    return np.array(samples)

In [ ]:
sample1 = karplus_strong(wavetable, 2 * fs)
Audio(sample1, rate=8000)

# drums


In [ ]:
def karplus_strong_drum(wavetable, n_samples, prob):
    """Synthesizes a new waveform from an existing wavetable, modifies last sample by averaging."""
    samples = []
    current_sample = 0
    previous_value = 0
    while len(samples) < n_samples:
        r = np.random.binomial(1, prob)
        sign = float(r == 1) * 2 - 1
        wavetable[current_sample] = sign * 0.5 * (wavetable[current_sample] + previous_value)
        samples.append(wavetable[current_sample])
        previous_value = samples[-1]
        current_sample += 1
        current_sample = current_sample % wavetable.size
    return np.array(samples)

In [ ]:
wavetable_size = fs // 40 
wavetable = np.ones(wavetable_size)

In [ ]:
sample1 = karplus_strong_drum(wavetable, 1 * fs, 0.3)

In [ ]:
Audio(sample1, rate=fs)

In [ ]:
bs = np.arange(0, 1.1, 0.1)

In [ ]:
for b in bs:
    wavetable = np.ones(wavetable_size)
    sample = karplus_strong_drum(wavetable, 1 * fs, b)
    display(Audio(sample, rate=fs))

In [ ]:
fs = 20000
for freq in [20, 55, 110, 220, 440, 880, 1288]:
    wavetable_size = fs // freq 
    wavetable = np.ones(wavetable_size)
    sample = karplus_strong_drum(wavetable, 2 * fs, 0)
    display(Audio(sample, rate=fs))

In [ ]:
## Modifying the algorithm for longer delays

In [ ]:
def karplus_strong_decay(wavetable, n_samples, stretch_factor):
    """Synthesizes a new waveform from an existing wavetable, modifies last sample by averaging.
    Uses a stretch_factor to control for decay."""
    samples = []
    current_sample = 0
    previous_value = 0
    while len(samples) < n_samples:
        r = np.random.binomial(1, 1 - 1/stretch_factor)
        if r == 0:
            wavetable[current_sample] =  0.5 * (wavetable[current_sample] + previous_value)
        samples.append(wavetable[current_sample])
        previous_value = samples[-1]
        current_sample += 1
        current_sample = current_sample % wavetable.size
    return np.array(samples)

In [ ]:
fs = 8000

In [ ]:
stretch_factors = [1, 2.1, 3.5, 4, 8]

In [ ]:
freq = 110

In [ ]:
waveforms = []
for ind, stretch_factor in enumerate(stretch_factors):
    wavetable_size = fs // int(freq)
    wavetable = (2 * np.random.randint(0, 2, wavetable_size) - 1).astype(np.float)
    sample = karplus_strong_decay(wavetable, 2 * fs, stretch_factor)
    waveforms.append(sample)

In [ ]:
for waveform in waveforms:
    display(Audio(waveform, rate=fs))

In [ ]:
plot.figure(figsize=(10, 7))
for ind, (waveform, stretch_factor) in enumerate(zip(waveforms, stretch_factors)):
    plot.plot(waveform - 2 * ind, label='{:.2f} stretch'.format(stretch_factor))
plot.legend(loc='upper right')
plot.title('full waveforms')

a = plot.axes([.5, .1, .4, .4], facecolor='white')
plot.yticks([])
for ind, (waveform, stretch_factor) in enumerate(zip(waveforms, stretch_factors)):
    plot.plot(waveform - 2 * ind, label='{:.2f} stretch'.format(stretch_factor))
plot.xlim(0, 1500)
plot.title('first samples')
plot.tight_layout()

In [ ]:
wavetable_size = fs // int(freq)
wavetable = make_sine_wavetable(wavetable_size, [0.3, 0.5, 0.3], [0, 0.5, 0], [1, 3, 9])
sample = karplus_strong_decay(wavetable, 5 * fs, stretch_factor=10)
plot.subplot(211)
plot.plot(sample)
plot.subplot(212)
plot.plot(sample)
plot.xlim(0, 1000)

Audio(sample, rate=fs)